In [ ]:
import os
import glob
import time
import re
import sys
sys.path.insert(0, "..")

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
from skimage import io

# Per yolov, ricordati di mettere a posto i magheggi del codice

In [ ]:
def get_all_needed(folder_name: str):
    name_list = ["{:03d}".format(i) for i in range(200)]
    image_array = [io.imread(os.path.join("..", "img",
                    "{}.jpg".format(name))) for name in name_list]
    track_array = [np.loadtxt(os.path.join("..", "img",
                            "{}_half.txt".format(name))) for name in name_list]
    
    track_file = [np.loadtxt(os.path.join(folder_name,
                                          "{name}.txt".format(name=name))) for name in name_list]
    
    return name_list, image_array, track_array, track_file

def false_negative(track_array: np.array, 
                   true_array: np.array):
    array_true = []
    array_false_negative = []
    for i in range(len(track_array)):
        true = []
        false_negative = []
        #image non empty
        if len(true_array[i]) != 0:
            #something found
            if len(track_array[i]) != 0:
                #more than 1 blob found
                if not np.isscalar(track_array[i][0]):
                    a_blob_i = [i for i in range(len(track_array[i]))]
                    if not np.isscalar(true_array[i][0]):
                            for blob in true_array[i]:
                                y, x, r = blob
                                check = False                
                                for a_i in a_blob_i:
                                    y_a, x_a, r_a = track_array[i][a_i]
                                    d2 = (x - x_a)**2 + (y - y_a)**2
                                    if d2 <= (2*r_a)**2:
                                        check = True
                                        break
                                if check:
                                    true.append(blob)
                                    a_blob_i.remove(a_i)
                                else:
                                    false_negative.append(blob)
                    else:
                        y, x, r = true_array[i]
                        check = False                
                        for a_i in a_blob_i:
                            y_a, x_a, r_a = track_array[i][a_i]
                            d2 = (x - x_a)**2 + (y - y_a)**2
                            if d2 <= (2*r_a)**2:
                                check = True
                                break
                        if check:
                            true.append(true_array[i])
                            a_blob_i.remove(a_i)
                        else:
                            false_negative.append(true_array[i])

                else:
                    if not np.isscalar(true_array[i][0]):
                        true_i = [i for i in range(len(true_array[i]))]
                        for t_i in true_i:
                            y, x, r = true_array[i][t_i]
                            y_a, x_a, r_a = track_array[i]
                            d2 = (x - x_a)**2 + (y - y_a)**2
                            if d2 <= (2*r_a)**2:
                                true.append(true_array[i][t_i])
                                true_i.remove(t_i)
                                break
                        for t_i in true_i:
                            false_negative.append(true_array[i][t_i])
                    else:
                        y, x, r = true_array[i]
                        y_a, x_a, r_a = track_array[i]
                        d2 = (x - x_a)**2 + (y - y_a)**2
                        if d2 <= (2*r_a)**2:
                            true.append(true_array[i])
                        else:
                            false_negative.append(true_array[i])

                array_true.append(true)
                array_false_negative.append(false_negative)

        else:
            array_true.append(true_array[i])
            array_false_negative.append(false_negative)
            
    return array_true, array_false_negative

def false_positive(track_array: np.array, 
                   true_array: np.array):
    array_true = []
    array_false_positive = []
    for i in range(len(track_array)):
        true = []
        false_positive = []
        if len(true_array[i]) != 0:
            if len(track_array[i]) != 0:
                if not np.isscalar(track_array[i][0]):
                    if not np.isscalar(true_array[i][0]):
                        blob_j = [j for j in range(len(true_array[i]))]
                        for a_blob in track_array[i]:
                            y_a, x_a, r_a = a_blob
                            check = False
                            for b_i in blob_j:
                                y, x, r = true_array[i][b_i]
                                d2 = (x - x_a)**2 + (y - y_a)**2
                                if d2 <= (2*r_a)**2:
                                    check = True
                                    true.append(true_array[i][b_i])
                                    blob_j.remove(b_i)
                                    break
                            if not check:
                                false_positive.append(a_blob)
                    else:
                        a_blob_i = [i for i in range(len(track_array[i]))]
                        for a_i in a_blob_i:
                            y_a, x_a, r_a = track_array[i][a_i]
                            y, x, r = true_array[i]
                            d2 = (x - x_a)**2 + (y - y_a)**2
                            if d2 <= (2*r_a)**2:
                                true.append(true_array[i])
                                a_blob_i.remove(a_i)
                                break
                        for a_i in a_blob_i:
                            false_positive.append(track_array[i][a_i])                       

                else:
                    if not np.isscalar(true_array[i][0]):
                        blob_j = [j for j in range(len(true_array[i]))]
                        y_a, x_a, r_a = track_array[i]
                        check = False
                        for b_i in blob_j:
                            y, x, r = true_array[i][b_i]
                            d2 = (x - x_a)**2 + (y - y_a)**2
                            if d2 <= (2*r_a)**2:
                                check = True
                                true.append(true_array[i][b_i])
                                blob_j.remove(b_i)
                                break
                        if not check:
                            false_positive.append(track_array[i])
                    else:
                        y_a, x_a, r_a = track_array[i]
                        y, x, r = true_array[i]
                        d2 = (x - x_a)**2 + (y - y_a)**2
                        if d2 <= (2*r_a)**2:
                            true.append(true_array[i])
                        else:
                            false_positive.append(track_array[i])

                array_true.append(true)
                array_false_positive.append(false_positive)
        else:
            array_true.append(true_array[i])
            array_false_positive.append(track_array[i])
     
    return array_true, array_false_positive

def number_track_plot(text_track_array: np.array,
                      true_track_array: np.array,
                      false_negative_array: np.array,
                      false_positive_array: np.array,
                      name_list: list,
                      algo_name: str):

    controll = []
    true_track = []
    false_negative_track = []
    false_positive_track = []
    for i in range(len(name_list)):
        false_negative = len(false_negative_array[i])
        false_positive = len(false_positive_array[i])
        true_found = len(true_track_array[i])
        total_tracks = len(text_track_array[i]) + false_positive
        
        if total_tracks != 0:
            error_true_neg = np.round((true_found/total_tracks)*100, 2)
            error_negative = np.round((false_negative/total_tracks)*100, 2)
            error_positive = np.round((false_positive/total_tracks)*100, 2)
            total = error_true_neg + error_negative + error_positive
        
        else:
            error_true_neg = 0
            error_negative = 0
            error_positive = 0
            total = 100          
            
        controll.append(total)
        true_track.append(error_true_neg)
        false_negative_track.append(error_negative)
        false_positive_track.append(error_positive)
    
    mean_true = np.mean(np.asarray(true_track))
    std_true = np.std(np.asarray(true_track))
    mean_positive = np.mean(np.asarray(false_positive_track))
    std_positive = np.std(np.asarray(false_positive_track))
    mean_negative = np.mean(np.asarray(false_negative_track))
    std_negative = np.std(np.asarray(false_negative_track))
    name = [int(name) for name in name_list]

    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 5))
    fig.suptitle("{} histogram".format(algo_name), y=1.05)

    ax1.hist(true_track, bins=15, color='b', label="True tracks")
    ax1.axvline(np.median(true_track), color="black", linestyle="--", alpha=0.25)
    ax1.set_xlim((0, 100))
    ax1.set_xlabel("Percentage")
    ax1.set_ylabel("Number of images")
    ax1.set_title("True tracks mean {:.2f} +- {:.2f}".format(np.median(true_track),
                                                             np.std(true_track)), y=1.05)
    ax1.legend()
    
    ax2.hist(false_negative_track, bins=10, color='g', label="False negative")
    ax2.axvline(np.median(false_negative_track), color="black", linestyle="--", alpha=0.25)
    ax2.set_xlim((0, 100))
    ax2.set_xlabel("Percentage")
    ax2.set_ylabel("Number of images")
    ax2.set_title("False negative tracks mean {:.2f} +- {:.2f}".format(np.median(false_negative_track),
                                                                       np.std(false_negative_track)), y=1.05)
    ax2.legend()
    
    ax3.hist(false_positive_track, bins=15, color='r', label="False positive")
    ax3.axvline(np.median(false_positive_track), color="black", linestyle="--", alpha=0.25)
    ax3.set_xlim((0, 100))
    ax3.set_xlabel("Percentage")
    ax3.set_ylabel("Number of images")
    ax3.set_title("False positive {:.2f} +- {:.2f}".format(np.median(false_positive_track),
                                                           np.std(false_positive_track)), y=1.05)
    ax3.legend()
    
    plt.tight_layout()
    #plt.savefig("Neural_network_histogram", bbox_inches='tight')
    plt.show()
    plt.close()
    
    return true_track, false_negative_track, false_positive_track

In [ ]:
names, images, texts, yol_track_array = get_all_needed(os.path.join("Log_result", "Fixed", "First"))
true_from_fals_neg, false_negative_track = false_negative(yol_track_array, texts)
true_from_fals_pos, false_positive_track = false_positive(yol_track_array, texts)

In [ ]:
yol_correct, yol_missed, yol_wrong = number_track_plot(texts,
                                                       true_from_fals_pos,
                                                       false_negative_track,
                                                       false_positive_track,
                                                       names,
                                                       "Neural Network")

In [ ]:
image_239 = io.imread(os.path.join("..", "img", "239.jpg"))
print(image_239.shape)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 14))
fig.suptitle("Visual Representation: green true tracks; blue false negative tracks; red false positive tracks", y=0.65)

ax1.imshow(images[0], cmap='gray')
if len(yol_track_array[0]) != 0:
    if len(np.asarray(true_from_fals_pos[0])) != 0:
        ax1.scatter(np.asarray(true_from_fals_pos[0])[:, 1],
                np.asarray(true_from_fals_pos[0])[:, 0],
                s=np.asarray(true_from_fals_pos[0])[:, 2],
                color='green')
if len(false_negative_track[0]) != 0:
    ax1.scatter(np.asarray(false_negative_track[0][0][1]),
                np.asarray(false_negative_track[0][0][0]),
                s=np.asarray(false_negative_track[0][0][2]),
                color='blue')
if len(false_positive_track[0]) != 0:
    ax1.scatter(np.asarray(false_positive_track[0])[:, 1],
                np.asarray(false_positive_track[0])[:, 0],
                s=np.asarray(false_positive_track[0])[:, 2],
                color='red')
ax1.set_title("Image {}: {} real; {} found; {} errors".format(names[0],
                                                              len(texts[0]),
                                                              len(true_from_fals_pos[0]),
                                                              len(false_positive_track[0])))
ax2.imshow(images[163], cmap='gray')
if len(yol_track_array[163]) != 0:
    ax2.scatter(np.asarray(true_from_fals_pos[163])[:, 1],
                np.asarray(true_from_fals_pos[163])[:, 0],
                s=np.asarray(true_from_fals_pos[163])[:, 2],
                color='green')
if len(false_negative_track[163]) != 0:
    ax2.scatter(np.asarray(false_negative_track[163])[:, 1],
                np.asarray(false_negative_track[163])[:, 0],
                s=np.asarray(false_negative_track[163])[:, 2],
                color='blue')
if len(false_positive_track[163]) != 0:
    ax2.scatter(np.asarray(false_positive_track[163])[:, 1],
                np.asarray(false_positive_track[163])[:, 0],
                s=np.asarray(false_positive_track[163])[:, 2],
                color='red')
ax2.set_xlim(0, 2592)
ax2.set_title("Image {}: {} real; {} found; {} errors".format(names[163],
                                                              len(texts[163]),
                                                              len(true_from_fals_pos[163]),
                                                              len(false_positive_track[163])))

ax3.imshow(images[92], cmap='gray')
if len(yol_track_array[92]) != 0:
    ax3.scatter(np.asarray(true_from_fals_pos[92])[:, 1],
                np.asarray(true_from_fals_pos[92])[:, 0],
                s=np.asarray(true_from_fals_pos[92])[:, 2],
                color='green')
if len(false_negative_track[92]) != 0:
    ax3.scatter(np.asarray(false_negative_track[92])[:, 1],
                np.asarray(false_negative_track[92])[:, 0],
                s=np.asarray(false_negative_track[92])[:, 2],
                color='blue')
if len(false_positive_track[92]) != 0:
    ax3.scatter(np.asarray(false_positive_track[92])[:, 1],
                np.asarray(false_positive_track[92])[:, 0],
                s=np.asarray(false_positive_track[92])[:, 2],
                color='red')
ax3.set_xlim(0, 2592)
ax3.set_title("Image {}: {} real; {} found; {} errors".format(names[92],
                                                              len(texts[92]),
                                                              len(true_from_fals_pos[92]),
                                                              len(false_positive_track[92])))


plt.savefig("Esempio_rete_neurale", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
#creazione dei gruppi: vuote, 1-21, 21-100, 500+
def create_group(text_array: np.array,
                 true_track_array: np.array,
                 negative_track_array: np.array,
                 positive_track_Array: np.array):
    
    group_1 = []
    #group_2 = []
    #group_3 = []
    #group_4 = []
    for indx in range(len(text_array)):
        text = text_array[indx]
        true = np.asarray(true_track_array[indx])
        negative = np.asarray(negative_track_array[indx])
        positive = np.asarray(positive_track_Array[indx])
        
        if len(text) != 0:
            group_1.append(np.array([text, true, negative, positive]))
        
        #if (len(text) >= 1) and (len(text) <= 21):
        #    group_2.append(np.array([text, true, negative, positive]))
            
        #if (len(text) > 21) and (len(text) <= 100):
        #    group_3.append(np.array([text, true, negative, positive]))
            
        #if len(text) > 100:
        #    group_4.append(np.array([text, true, negative, positive]))
        
    #group_collection = np.array([group_1, group_2, group_3, group_4])
       
    return group_1

def plotting_histogram(group: np.array):
    
    true_percent = []
    negative_percent = []
    positive_percent = []
    for element in group:
        total = len(element[0]) + len(element[-1])
        try:
            true = int((len(element[1])/total)*100)
            true_percent.append(true)
            
            negative = int((len(element[2])/total)*100)
            negative_percent.append(negative)
            
            positive = int((len(element[-1])/total)*100)
            positive_percent.append(positive)
        except ZeroDivisionError:
            true_percent.append(len(element[1]))
            negative_percent.append(len(element[2]))
            positive_percent.append(len(element[-1]))
            
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 5))
    fig.suptitle("Laplacian of Gaussian histogram", y=1)

    ax1.hist(true_percent, bins=10, color='b', edgecolor='black', linewidth=1.2)
    ax1.axvline(np.median(true_percent), color="black", linestyle="--", alpha=0.25)
    ax1.set_xlim((0, 100))
    ax1.set_xlabel("Percentage")
    ax1.set_ylabel("Number of images")
    ax1.set_title("True tracks mean {:.2f} +- {:.2f}".format(np.median(true_percent),
                                                             np.std(true_percent)), y=1.05)
    
    ax2.hist(negative_percent, bins=10, color='g', edgecolor='black', linewidth=1.2)
    ax2.axvline(np.median(negative_percent), color="black", linestyle="--", alpha=0.50)
    ax2.set_xlim((0, 100))
    ax2.set_xlabel("Percentage")
    ax2.set_ylabel("Number of images")
    ax2.set_title("False negative tracks mean {:.2f} +- {:.2f}".format(np.median(negative_percent),
                                                                       np.std(negative_percent)), y=1.05)
    
    ax3.hist(positive_percent, bins=10, color='r', edgecolor='black', linewidth=1.2)
    ax3.axvline(np.median(positive_percent), color="black", linestyle="--", alpha=0.50)
    ax3.set_xlim((0, 100))
    ax3.set_xlabel("Percentage")
    ax3.set_ylabel("Number of images")
    ax3.set_title("False positive {:.2f} +- {:.2f}".format(np.median(positive_percent),
                                                           np.std(positive_percent)), y=1.05)
    
    plt.tight_layout()
    plt.savefig("Log_histogram_group", bbox_inches='tight')
    plt.show()
    plt.close()


In [ ]:
all_group = create_group(texts,
                         true_from_fals_pos,
                         false_negative_track,
                         false_positive_track)

#counter = 1
#for group in all_group:
plotting_histogram(all_group)
#counter += 1

In [ ]:
import matplotlib.pyplot as plt


fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 5))
fig.suptitle("Neural Network histogram\n Group 1, 40 images", y=1.02)

ax1.bar(0, 40, width=2, align = "center", color= "green")
ax1.axvline(0, color="black", linestyle="--", alpha=0.25)
ax1.set_xlim((-2, 100))
ax1.set_xlabel("Percentage")
ax1.set_ylabel("Number of images")
ax1.set_title("True tracks mean\n 0 +- 0", y=1.05)

ax2.bar(0, 40, width=2, align = "center", color= "blue")
ax2.axvline(0, color="black", linestyle="--", alpha=0.25)
ax2.set_xlim((-2, 100))
ax2.set_xlabel("Percentage")
ax2.set_ylabel("Number of images")
ax2.set_title("False negative tracks\n 0 +- 0", y=1.05)

ax3.bar(0, 40, width=2, align = "center", color= "red")
ax3.axvline(0, color="black", linestyle="--", alpha=0.25)
ax3.set_xlim((-2, 100))
ax3.set_xlabel("Percentage")
ax3.set_ylabel("Number of images")
ax3.set_title("False positive tracks\n 0 +- 0", y=1.05)

plt.tight_layout()
plt.savefig("Yolo_first_group", bbox_inches='tight')
plt.show()
plt.close()

# Per threshold

In [ ]:
def get_all_needed(folder_name: str):
    name_list = ["{:03d}".format(i) for i in range(240)]
    image_array = [io.imread(os.path.join("..", "img",
                    "{}.jpg".format(name))) for name in name_list]
    track_array = [np.loadtxt(os.path.join("..", "img",
                            "{}_half.txt".format(name))) for name in name_list]
    
    track_file = [np.loadtxt(os.path.join(folder_name,
                                          "{name}.txt".format(name=name))) for name in name_list]
    
    return name_list, image_array, track_array, track_file

def false_negative(track_array: np.array, 
                   true_array: np.array):
    array_true = []
    array_false_negative = []
    for i in range(len(track_array)):
        true = []
        false_negative = []
        if len(true_array[i]) != 0:
            if len(track_array[i]) != 0:
                if not np.isscalar(track_array[i][0]):
                    a_blob_i = [i for i in range(len(track_array[i]))]
                    if not np.isscalar(true_array[i][0]):
                            for blob in true_array[i]:
                                y, x, r = blob
                                check = False                
                                for a_i in a_blob_i:
                                    y_a, x_a, r_a, area = track_array[i][a_i]
                                    d2 = (x - x_a)**2 + (y - y_a)**2
                                    if d2 <= (r)**2:
                                        check = True
                                        break
                                if check:
                                    true.append(track_array[i][a_i])
                                    a_blob_i.remove(a_i)
                                else:
                                    false_negative.append(blob)
                    else:
                        y, x, r = true_array[i]
                        check = False                
                        for a_i in a_blob_i:
                            y_a, x_a, r_a, area = track_array[i][a_i]
                            d2 = (x - x_a)**2 + (y - y_a)**2
                            if d2 <= (r)**2:
                                check = True
                                break
                        if check:
                            true.append(track_array[i][a_i])
                            a_blob_i.remove(a_i)
                        else:
                            false_negative.append(blob)

                else:
                    if not np.isscalar(true_array[i][0]):
                        true_i = [i for i in range(len(true_array[i]))]
                        for t_i in true_i:
                            y, x, r = true_array[i][t_i]
                            y_a, x_a, r_a, area = track_array[i]
                            d2 = (x - x_a)**2 + (y - y_a)**2
                            if d2 <= (r)**2:
                                true.append(track_array[i])
                                true_i.remove(t_i)
                                break
                        for t_i in true_i:
                            false_negative.append(true_array[i][t_i])
                    else:
                        y, x, r = true_array[i]
                        y_a, x_a, r_a, area = track_array[i]
                        d2 = (x - x_a)**2 + (y - y_a)**2
                        if d2 <= (r)**2:
                            true.append(track_array[i])
                        else:
                            false_negative.append(true_array[i])

                array_true.append(true)
                array_false_negative.append(false_negative)

        else:
            array_true.append(true_array[i])
            array_false_negative.append(false_negative)
            
    return array_true, array_false_negative


def false_positive(track_array: np.array, 
                   true_array: np.array):
    array_true = []
    array_false_positive = []
    for i in range(len(track_array)):
        true = []
        false_positive = []
        if len(true_array[i]) != 0:
            if len(track_array[i]) != 0:
                if not np.isscalar(track_array[i][0]):
                    if not np.isscalar(true_array[i][0]):
                        blob_j = [j for j in range(len(true_array[i]))]
                        for a_blob in track_array[i]:
                            y_a, x_a, r_a, area = a_blob
                            check = False
                            for b_i in blob_j:
                                y, x, r = true_array[i][b_i]
                                d2 = (x - x_a)**2 + (y - y_a)**2
                                if d2 <= (r)**2:
                                    check = True
                                    true.append(a_blob)
                                    blob_j.remove(b_i)
                                    break
                            if not check:
                                false_positive.append(a_blob)
                    else:
                        a_blob_i = [i for i in range(len(track_array[i]))]
                        for a_i in a_blob_i:
                            y_a, x_a, r_a, area = track_array[i][a_i]
                            y, x, r = true_array[i]
                            d2 = (x - x_a)**2 + (y - y_a)**2
                            if d2 <= (r)**2:
                                true.append(track_array[i][a_i])
                                a_blob_i.remove(a_i)
                                break
                        for a_i in a_blob_i:
                            false_positive.append(track_array[i][a_i])                       

                else:
                    if not np.isscalar(true_array[i][0]):
                        blob_j = [j for j in range(len(true_array[i]))]
                        y_a, x_a, r_a, area = track_array[i]
                        check = False
                        for b_i in blob_j:
                            y, x, r = true_array[i][b_i]
                            d2 = (x - x_a)**2 + (y - y_a)**2
                            if d2 <= (r)**2:
                                check = True
                                true.append(track_array[i])
                                blob_j.remove(b_i)
                                break
                        if not check:
                            false_positive.append(track_array[i])
                    else:
                        y_a, x_a, r_a, area = track_array[i]
                        y, x, r = true_array[i]
                        d2 = (x - x_a)**2 + (y - y_a)**2
                        if d2 <= (r)**2:
                            true.append(track_array[i])
                        else:
                            false_positive.append(track_array[i])

                array_true.append(true)
                array_false_positive.append(false_positive)
        else:
            array_true.append(true_array[i])
            array_false_positive.append(track_array[i])
     
    return array_true, array_false_positive

def number_track_plot(text_track_array: np.array,
                      true_track_array: np.array,
                      false_negative_array: np.array,
                      false_positive_array: np.array,
                      name_list: list,
                      algo_name: str):

    controll = []
    true_track = []
    false_negative_track = []
    false_positive_track = []
    for i in range(len(name_list)):
        false_negative = len(false_negative_array[i])
        false_positive = len(false_positive_array[i])
        true_found = len(true_track_array[i])
        total_tracks = len(text_track_array[i]) + false_positive
        
        if total_tracks != 0:
            error_true_neg = np.round((true_found/total_tracks)*100, 2)
            error_negative = np.round((false_negative/total_tracks)*100, 2)
            error_positive = np.round((false_positive/total_tracks)*100, 2)
            total = error_true_neg + error_negative + error_positive
        
        else:
            error_true_neg = 0
            error_negative = 0
            error_positive = 0
            total = 100          
            
        controll.append(total)
        true_track.append(error_true_neg)
        false_negative_track.append(error_negative)
        false_positive_track.append(error_positive)
    
    mean_true = np.mean(np.asarray(true_track))
    std_true = np.std(np.asarray(true_track))
    mean_positive = np.mean(np.asarray(false_positive_track))
    std_positive = np.std(np.asarray(false_positive_track))
    mean_negative = np.mean(np.asarray(false_negative_track))
    std_negative = np.std(np.asarray(false_negative_track))
    name = [int(name) for name in name_list]

    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 5))
    fig.suptitle("{} histogram".format(algo_name), y=1.05)

    ax1.hist(true_track, bins=15, color='b', label="True tracks")
    ax1.axvline(np.median(true_track), color="black", linestyle="--", alpha=0.25)
    ax1.set_xlim((0, 100))
    ax1.set_xlabel("Percentage")
    ax1.set_ylabel("Number of images")
    ax1.set_title("True tracks mean {:.2f} +- {:.2f}".format(np.median(true_track),
                                                             np.std(true_track)), y=1.05)
    ax1.legend()
    
    ax2.hist(false_negative_track, bins=10, color='g', label="False negative")
    ax2.axvline(np.median(false_negative_track), color="black", linestyle="--", alpha=0.25)
    ax2.set_xlim((0, 100))
    ax2.set_xlabel("Percentage")
    ax2.set_ylabel("Number of images")
    ax2.set_title("False negative tracks mean {:.2f} +- {:.2f}".format(np.median(false_negative_track),
                                                                       np.std(false_negative_track)), y=1.05)
    ax2.legend()
    
    ax3.hist(false_positive_track, bins=15, color='r', label="False positive")
    ax3.axvline(np.median(false_positive_track), color="black", linestyle="--", alpha=0.25)
    ax3.set_xlim((0, 100))
    ax3.set_xlabel("Percentage")
    ax3.set_ylabel("Number of images")
    ax3.set_title("False positive {:.2f} +- {:.2f}".format(np.median(false_positive_track),
                                                           np.std(false_positive_track)), y=1.05)
    ax3.legend()
    
    plt.tight_layout()
    #plt.savefig("Threshold_histogram", bbox_inches='tight')
    plt.show()
    plt.close()
    
    return true_track, false_negative_track, false_positive_track

In [ ]:
names, images, texts, track_array = get_all_needed("results")
true_from_fals_neg, false_negative_track = false_negative(track_array, texts)
true_from_fals_pos, false_positive_track = false_positive(track_array, texts)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(7, 7))
fig.suptitle("Visual Representation: blue true tracks; green false negative tracks; red false positive tracks", y=0.85)

ax1.imshow(images[0], cmap='gray')
if len(track_array[0]) != 0:
    if len(np.asarray(true_from_fals_pos[0])) != 0:
        ax1.scatter(np.asarray(true_from_fals_pos[0])[:, 1],
                np.asarray(true_from_fals_pos[0])[:, 0],
                s=np.asarray(true_from_fals_pos[0])[:, 2],
                color='blue')
if len(false_negative_track[0]) != 0:
    ax1.scatter(np.asarray(false_negative_track[0][0][1]),
                np.asarray(false_negative_track[0][0][0]),
                s=np.asarray(false_negative_track[0][0][2]),
                color='green')
if len(false_positive_track[0]) != 0:
    ax1.scatter(np.asarray(false_positive_track[0])[:, 1],
                np.asarray(false_positive_track[0])[:, 0],
                s=np.asarray(false_positive_track[0])[:, 2],
                color='red')
    
#plt.savefig("Tracce_mostrate", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
thresh_correct, thresh_missed, thresh_wrong = number_track_plot(texts,
                                                        true_from_fals_pos,
                                                       false_negative_track,
                                                       false_positive_track,
                                                       names,
                                                       "Threshold")

In [ ]:
fig, (ax1, ax2, ax4) = plt.subplots(1, 3, figsize=(14, 9))
fig.suptitle("Visual Representation: green true tracks; blue false negative tracks; red false positive tracks", y=0.7)

ax1.imshow(images[0], cmap='gray')
if len(track_array[0]) != 0:
    if len(np.asarray(true_from_fals_pos[0])) != 0:
        ax1.scatter(np.asarray(true_from_fals_pos[0])[:, 1],
                np.asarray(true_from_fals_pos[0])[:, 0],
                s=np.asarray(true_from_fals_pos[0])[:, 2],
                color='green')
if len(false_negative_track[0]) != 0:
    ax1.scatter(np.asarray(false_negative_track[0][0][1]),
                np.asarray(false_negative_track[0][0][0]),
                s=np.asarray(false_negative_track[0][0][2]),
                color='blue')
if len(false_positive_track[0]) != 0:
    ax1.scatter(np.asarray(false_positive_track[0])[:, 1],
                np.asarray(false_positive_track[0])[:, 0],
                s=np.asarray(false_positive_track[0])[:, 2],
                color='red')
ax1.set_title("Image {}: {} real; {} found; {} errors".format(names[0],
                                                              len(texts[0]),
                                                              len(true_from_fals_pos[0]),
                                                              len(false_positive_track[0])))

ax2.imshow(images[163], cmap='gray')
if len(track_array[163]) != 0:
    ax2.scatter(np.asarray(true_from_fals_pos[163])[:, 1],
                np.asarray(true_from_fals_pos[163])[:, 0],
                s=np.asarray(true_from_fals_pos[163])[:, 2],
                color='green')
if len(false_negative_track[163]) != 0:
    ax2.scatter(np.asarray(false_negative_track[163])[:, 1],
                np.asarray(false_negative_track[163])[:, 0],
                s=np.asarray(false_negative_track[163])[:, 2],
                color='blue')
if len(false_positive_track[163]) != 0:
    ax2.scatter(np.asarray(false_positive_track[163])[:, 1],
                np.asarray(false_positive_track[163])[:, 0],
                s=np.asarray(false_positive_track[163])[:, 2],
                color='red')
ax2.set_xlim(0, 2592)
ax2.set_title("Image {}: {} real; {} found; {} errors".format(names[163],
                                                              len(texts[163]),
                                                              len(true_from_fals_pos[163]),
                                                              len(false_positive_track[163])))

ax4.imshow(images[92], cmap='gray')
if len(track_array[92]) != 0:
    ax4.scatter(np.asarray(true_from_fals_pos[92])[:, 1],
                np.asarray(true_from_fals_pos[92])[:, 0],
                s=np.asarray(true_from_fals_pos[92])[:, 2],
                color='green')
if len(false_negative_track[92]) != 0:
    ax4.scatter(np.asarray(false_negative_track[92])[:, 1],
                np.asarray(false_negative_track[92])[:, 0],
                s=np.asarray(false_negative_track[92])[:, 2],
                color='blue')
if len(false_positive_track[163]) != 0:
    ax4.scatter(np.asarray(false_positive_track[92])[:, 1],
                np.asarray(false_positive_track[92])[:, 0],
                s=np.asarray(false_positive_track[92])[:, 2],
                color='red')
ax4.set_xlim(0, 2592)
ax4.set_title("Image {}: {} real; {} found; {} errors".format(names[92],
                                                              len(texts[92]),
                                                              len(true_from_fals_pos[92]),
                                                              len(false_positive_track[92])))

plt.savefig("Esempio_threshold", bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
#creazione dei gruppi: vuote, 1-21, 21-100, 500+
def create_group(text_array: np.array,
                 true_track_array: np.array,
                 negative_track_array: np.array,
                 positive_track_Array: np.array):
    
    group_1 = []
    #group_2 = []
    #group_3 = []
    #group_4 = []
    for indx in range(len(text_array)):
        text = text_array[indx]
        true = np.asarray(true_track_array[indx])
        negative = np.asarray(negative_track_array[indx])
        positive = np.asarray(positive_track_Array[indx])
        
        if len(text) != 0:
            group_1.append(np.array([text, true, negative, positive]))
        
        #if (len(text) >= 1) and (len(text) <= 21):
        #else:
        #    group_2.append(np.array([text, true, negative, positive]))
            
        #if (len(text) > 21) and (len(text) <= 100):
        #    group_3.append(np.array([text, true, negative, positive]))
            
        #if len(text) > 100:
        #    group_4.append(np.array([text, true, negative, positive]))
        
    #group_collection = np.array([group_1, group_2, group_3, group_4])
       
    return group_1

def plotting_histogram(group: np.array):
    
    true_percent = []
    negative_percent = []
    positive_percent = []
    for element in group:
        total = len(element[0]) + len(element[-1])
        try:
            true = int((len(element[1])/total)*100)
            true_percent.append(true)
            
            negative = int((len(element[2])/total)*100)
            negative_percent.append(negative)
            
            positive = int((len(element[-1])/total)*100)
            positive_percent.append(positive)
        except ZeroDivisionError:
            true_percent.append(len(element[1]))
            negative_percent.append(len(element[2]))
            positive_percent.append(len(element[-1]))
            
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 5))
    fig.suptitle("Threshold Algorithm histogram", y=1.0)

    ax1.hist(true_percent, bins=7, color='b', edgecolor='black', linewidth=1.2)
    ax1.axvline(np.median(true_percent), color="black", linestyle="--", alpha=0.25)
    ax1.set_xlim((0, 100))
    ax1.set_xlabel("Percentage")
    ax1.set_ylabel("Number of images")
    ax1.set_title("True tracks mean {:.2f} +- {:.2f}".format(np.median(true_percent),
                                                             np.std(true_percent)), y=1.05)
    
    ax2.hist(negative_percent, bins=7, color='g', edgecolor='black', linewidth=1.2)
    ax2.axvline(np.median(negative_percent), color="black", linestyle="--", alpha=0.50)
    ax2.set_xlim((0, 100))
    ax2.set_xlabel("Percentage")
    ax2.set_ylabel("Number of images")
    ax2.set_title("False negative tracks mean {:.2f} +- {:.2f}".format(np.median(negative_percent),
                                                                       np.std(negative_percent)), y=1.05)
    
    ax3.hist(positive_percent, bins=7, color='r', edgecolor='black', linewidth=1.2)
    ax3.axvline(np.median(positive_percent), color="black", linestyle="--", alpha=0.50)
    ax3.set_xlim((0, 100))
    ax3.set_xlabel("Percentage")
    ax3.set_ylabel("Number of images")
    ax3.set_title("False positive {:.2f} +- {:.2f}".format(np.median(positive_percent),
                                                           np.std(positive_percent)), y=1.05)
    
    plt.tight_layout()
    plt.savefig("Thresh_histogram_group", bbox_inches='tight')
    plt.show()
    plt.close()


In [ ]:
all_group = create_group(texts,
                         true_from_fals_pos,
                         false_negative_track,
                         false_positive_track)

#counter = 1
#for group in all_group:
plotting_histogram(all_group)
#counter += 1

In [ ]:
import matplotlib.pyplot as plt


fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 5))
fig.suptitle("Threshold Algorithm histogram\n Group 1, 40 images", y=1.02)

ax1.bar(0, 40, width=2, align = "center", color= "green")
ax1.axvline(0, color="black", linestyle="--", alpha=0.25)
ax1.set_xlim((-2, 100))
ax1.set_xlabel("Percentage")
ax1.set_ylabel("Number of images")
ax1.set_title("True tracks mean\n 0 +- 0", y=1.05)

ax2.bar(0, 40, width=2, align = "center", color= "blue")
ax2.axvline(0, color="black", linestyle="--", alpha=0.25)
ax2.set_xlim((-2, 100))
ax2.set_xlabel("Percentage")
ax2.set_ylabel("Number of images")
ax2.set_title("False negative tracks\n 0 +- 0", y=1.05)

ax3.bar(0, 40, width=2, align = "center", color= "red")
ax3.axvline(0, color="black", linestyle="--", alpha=0.25)
ax3.set_xlim((-2, 100))
ax3.set_xlabel("Percentage")
ax3.set_ylabel("Number of images")
ax3.set_title("False positive tracks\n 0 +- 0", y=1.05)

plt.tight_layout()
plt.savefig("Thresh_first_group", bbox_inches='tight')
plt.show()
plt.close()